# [LangChain Crash Course - Build apps with language models](https://www.youtube.com/watch?v=LbT1yp6quS8)

## Overview
* Installation
* LLMs
* Prompt Templates
* Chains
* Agents and Tools
* Memory
* Document Loaders
* Indexes

## Installation

In [24]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext dotenv
%dotenv

## 1. LLMs

A generic interface for all LLMS. See all LLM providers: [https://python.langchain.com/en/latest/modules/models/llms/integrations.html](https://python.langchain.com/en/latest/modules/models/llms/integrations.html)


In [3]:
import os

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9) # model_name="text-davinci-003"
text = "What is a good company name for a company that sells colorful socks?"
print(llm(text))



Walk in Color Socks.


In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

from langchain import HuggingFaceHub

# https://huggingface.co/google/flan-t5-xl
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature": 0, "max_length": 64})

llm("Translate English to Spanish: How old are you?")

/Users/mattmasteller/Code/Learn/langchain/spike-langchain/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Cuántos aos tiene?'

## 2. Prompt Templates

LangChain facilitates prompt management and optimization.

Normally when you us an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompts, and only then send that to the LLM. 

In [5]:
llm("Can Barack Obama have a conversation with George Washington?")

'Barack Obama is a current president. George Washington was a past president.'

In [6]:
prompt = """Question: Can Barack Obama have a conversation with George Washington?

Let's think step by step.

Answer:"""

llm(prompt)

'Barack Obama was born in 1961. George Washington was born in 1799. So the final answer is no.'

In [7]:
from langchain import PromptTemplate

template = """Question: {question}

Let's think step by step.

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [8]:
prompt.format(question="Can Barack Obama have a conversation with George Washington?")

"Question: Can Barack Obama have a conversation with George Washington?\n\nLet's think step by step.\n\nAnswer:"

In [9]:
llm(prompt)

TypeError: Object of type PromptTemplate is not JSON serializable

## 3. Chains

Combine LLMs and Prompts into multi-step workflows.

In [10]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington?"

print(llm_chain.run(question))

Barack Obama was born in 1961. George Washington was born in 1799. So the final answer is no.


## 4. Agents and Tools

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done. 

When used correctly, Agents can be extremely powerful. In order to load Agents, you should understand the following concepts:

* Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
* LLM: The language model powering the agent.
* Agent: The agent to use.

Tools: [https://python.langchain.com/en/latest/modules/agents/tools.html](https://python.langchain.com/en/latest/modules/agents/tools.html)

Agent Types: [https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html)

In [12]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

In [13]:
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

In [14]:
agent.run("What year was the film 'Departed' with Leonardo DiCaprio released? What is the year raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out the year the film was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: "Departed" film
Observation: Page: The Departed
Summary: The Departed is a 2006 American epic crime thriller film directed by Martin Scorsese and written by William Monahan. It is both a remake of the 2002 Hong Kong film Infernal Affairs and also loosely based on the real-life Boston Winter Hill Gang; the character Colin Sullivan is based on the corrupt FBI agent John Connolly, while the character Frank Costello is based on Irish-American gangster and crime boss Whitey Bulger. The film stars Leonardo DiCaprio, Matt Damon, Jack Nicholson, and Mark Wahlberg, with Martin Sheen, Ray Winstone, Vera Farmiga, Alec Baldwin, Anthony Anderson and James Badge Dale in supporting roles.
The film takes place in Boston and the surrounding metro area, primarily in the city’s South End neighborhood. Irish Mob boss Frank Costello (

"The film 'Departed' with Leonardo DiCaprio was released in 2006 and the year raised to the 0.43 power is 26.30281917656938."

## 5. Memory

Add State to Chains and Agents.

Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [15]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [16]:
conversation.predict(input="Can we talk about AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:

> Finished chain.


' Absolutely! What would you like to know about AI?'

In [17]:
conversation.predict(input="I'm interested in Reinforcement Learning.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:  Absolutely! What would you like to know about AI?
Human: I'm interested in Reinforcement Learning.
AI:

> Finished chain.


' Sure! Reinforcement Learning is a type of machine learning algorithm that allows an AI agent to learn from its environment by taking actions and receiving rewards or punishments. It is used to solve complex problems that require trial and error. Would you like to know more about how it works?'

## 6. Document Loaders

Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into "documents" - a fancy way of saying "pieces of text". This module is aimed at making this easy.

[https://python.langchain.com/en/latest/modules/indexes/document_loaders.html](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html)

In [18]:
from langchain.document_loaders import NotionDirectoryLoader

loader = NotionDirectoryLoader("Notion_DB")

docs = loader.load()

## 7. Indexes

Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents

- Embeddings: An embedding is a numerical representation of a piece of information, for example, text, documents, images, audio, etc.
- Text Splitters: When you want to deal with long pieces of text, it is necessary to split up that text into chunks.
- Vectorstores: Vector databases store and index vector embeddings from NLP models to understand the meaning and context of strings of text, sentences, and whole documents for more accurate and relevant search results.

In [29]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("./data/state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [30]:
# Document Loader
from langchain.document_loaders import TextLoader

loader = TextLoader('./data/state_of_the_union.txt')
documents = loader.load()

In [32]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [33]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

#text = "This is a test document."
#query_result = embeddings.embed_query(text)
#doc_result = embeddings.embed_documents([text])

In [34]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [35]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [38]:
db.save_local("./data/faiss_index")
new_db = FAISS.load_local("./data/faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## End-to-end example

https://github.com/hwchase17/chat-langchain